# Reconhecimento de emoções em imagens usando o tensor flow

A tarefa consiste em criar uma rede neural usando a biblioteca TensorFlow, que consiga classificar com a melhor precisão possível as expressões faciais do conjunto de dados fornecido em anexo. O seu modelo deve conseguir classificar as imagens usando uma das 6 emoções a seguir: Raiva, Nojo, Medo, Feliz, Triste, Surpresa ou Neutro.
O TensorFlow tem suporte para treinamento via GPU e CPU. Como os computadores da faculdade não possuem GPU dedicada e nem meu próprio computador de trabalho, **o treinamento dos dados deve ser feito usando somente a CPU para que eu possa replicar o procedimento no meu computador no momento da correção.**
**Tenha em mente que o processo será demorado por conta do uso de somente CPU...**

## Dados

O banco de dados possui 3 colunas: emotions, pixels, Usage

**Emotions**: são as emoções das imagens: 0=Angry, 1=Disgust, 2=Fear, 3=Happy, 4=Sad, 5=Surprise, 6=Neutral

**Pixels**: é a imagem 64x64 (matriz de pixels ) já transformado de imagem em uma matriz de números. As imagens são em branco e preto portanto 

**Usage**: indica se é para ser usado como treinamento ou teste. Isso ja facilita a divisão dos dados de treinamento e teste =)

## Entrega do trabalho  

Cada integrante do grupo deverá entregar na tarefa do Teams um arquivo de notebook (.ipynb) contendo todo o código desenvolvido (células de código) e explicações de cada passo dado no projeto (células markdown). Caso algum integrante esqueça de entregar sua cópia do trabalho ele poderá ficar sem nota. Importante: todos os integrantes do grupo devem entregar o mesmo trabalho.  



## Grupo  

O trabalho será desenvolvido em grupos de 2 até 4 alunos. O nome dos integrantes deve constar numa célula de markdown no início do trabalho. Respeitem os tamanhos dos grupos. 

## FAQ  

**Trabalhos iguais em grupos diferentes**: Zero para ambos os grupos. Cuidado com o que vocês compartilham entre si.  

**Usei uma IA para fazer os códigos pra mim, por isso ficou igual ao do outro grupo**: Zero para ambos os grupos. Usem as IA (GPT e afins) com responsabilidade. Não sai copiando e colando qualquer coisa que eles respondem... Usem a cabeça...  

**Achei um desafio do Kagle igualzinho a esse e eu tomei base nele por isso ficou igual ao do meu colega**: Novamente, Não sai copiando e colando qualquer coisa que vocês acharem... Use trabalhos prontos como inspiração, não como resolução.  

**Segui um tutorial do YouTube, por isso ficou igual ao do outro grupo**: Zero  

**Trabalho encontrado na internet igual ao do grupo**: Zero  

## Grupo

- Murilo Ramalho da Mata
- Camila Gomes da Silva Casa
- Aline Favalli Ferreira
- Bianca Silva Henrique

In [1]:
%pip install pandas numpy tensorflow scikit-learn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# Importar bibliotecas necessárias
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf

### Expressões Faciais: [base dados](./ExpressoesFaciais.csv.csv)

In [3]:
# Carregar os dados
df = pd.read_csv('ExpressoesFaciais.csv')

In [4]:
# Verificar se o DataFrame foi carregado corretamente
print("Dados carregados:")
print(df.head())
print(f"Número total de amostras: {len(df)}")
print(df.info())

# Função para converter a string de pixels em um array numpy
def convert_pixels(pixel_string):
    pixel_array = np.array(pixel_string.split(), dtype='float')
    if pixel_array.size == 48 * 48:  # Ajustado para 48x48
        return pixel_array.reshape(48, 48, 1)
    else:
        print(f"Comprimento inesperado: {pixel_array.size}")
        return None

# Aplicar a conversão de pixels
df['pixels'] = df['pixels'].apply(lambda x: convert_pixels(x) if isinstance(x, str) else None)

# Remover registros inválidos
df.dropna(subset=['pixels'], inplace=True)

# Exibir informações após a conversão
print(f"Número total de amostras após a conversão: {len(df)}")
print(f"Classes únicas: {df['emotion'].unique()}")

# Separar características e rótulos
if len(df) > 0:
    X = np.array(df['pixels'].tolist())
    y = df['emotion'].values

    # Normalizar os valores de pixels para o intervalo [0, 1]
    X = X / 255.0

    # Dividir os dados em conjuntos de treinamento e teste
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

    # Construir o modelo de rede neural
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(48, 48, 1)),  # Ajustado para 48x48
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(7, activation='softmax')  # 7 classes de emoções
    ])

    # Compilar o modelo
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    # Treinar o modelo
    model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

    # Avaliar o modelo
    test_loss, test_accuracy = model.evaluate(X_test, y_test)
    print(f'Taxa de sucesso: {test_accuracy * 100:.2f}%')
else:
    print("Erro: DataFrame está vazio após a conversão.")

Dados carregados:
   emotion                                             pixels     Usage
0        0  70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...  Training
1        0  151 150 147 155 148 133 111 140 170 174 182 15...  Training
2        2  231 212 156 164 174 138 161 173 182 200 106 38...  Training
3        4  24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...  Training
4        6  4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...  Training
Número total de amostras: 35887
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35887 entries, 0 to 35886
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   emotion  35887 non-null  int64 
 1   pixels   35887 non-null  object
 2   Usage    35887 non-null  object
dtypes: int64(1), object(2)
memory usage: 841.2+ KB
None
Número total de amostras após a conversão: 35887
Classes únicas: [0 2 4 6 3 5 1]
Epoch 1/10
718/718 ━━━━━━━━━━━━━━━━━━━━ 15s 19ms/step - accuracy: 0.3153 - loss: 1.7051